### New attempt
Old accuracy: 81,83026200325937

In [30]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import torch.optim as optim
from torch.nn import functional as F

In [31]:
batch_size = 64
n_iters = 10000
num_epochs = 200
# Create CNN Model
n_timesteps = 46
n_outputs = 20

n = 199424 # number of samples less than 199424
train_pd = pd.read_csv('train.csv')

y_np = train_pd.V1.values[:n]
x_np = train_pd.loc[:,train_pd.columns != "V1"].values[:n, :]
y_np = y_np - 1
# y goes from 0 to 19

x_train_np, x_test_np, y_train_np, y_test_np = train_test_split(x_np, y_np, test_size=0.2) 

x_train_tensor = torch.from_numpy(x_train_np).type(torch.FloatTensor)
y_train_tensor = torch.from_numpy(y_train_np).type(torch.LongTensor)

x_test_tensor = torch.from_numpy(x_test_np).type(torch.FloatTensor)
y_test_tensor = torch.from_numpy(y_test_np).type(torch.LongTensor)

# Pytorch train and test sets
train = torch.utils.data.TensorDataset(x_train_tensor, y_train_tensor)
test = torch.utils.data.TensorDataset(x_test_tensor, y_test_tensor)

train_loader = DataLoader(train, batch_size = batch_size, shuffle = False)
test_loader = DataLoader(test, batch_size = batch_size, shuffle = False)

In [32]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [37]:
class LSTMClassifier(nn.Module):
    """Very simple implementation of LSTM-based time-series classifier."""
    
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.rnn = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.batch_size = None
        self.hidden = None
    
    def forward(self, x):
        x = x.unsqueeze(1)
        h0, c0 = self.init_hidden(x)
        out, (hn, cn) = self.rnn(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out
    
    def init_hidden(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        return h0, c0

In [ ]:
input_dim = 46   
hidden_dim = 512
layer_dim = 3
output_dim = 20
seq_dim = 128

lr = 0.0005
n_epochs = 1000
iterations_per_epoch = 500
best_acc = 0
patience, trials = 100, 0

model = LSTMClassifier(input_dim, hidden_dim, layer_dim, output_dim)

criterion = nn.CrossEntropyLoss()
opt = torch.optim.RMSprop(model.parameters(), lr=lr)

print('Start model training')

for epoch in range(1, n_epochs + 1):
    
    for i, (x_batch, y_batch) in enumerate(train_loader):
        model.train()

        opt.zero_grad()
        out = model(x_batch)
        loss = criterion(out, y_batch)
        loss.backward()
        opt.step()
    
    model.eval()
    correct, total = 0, 0
    for x_val, y_val in test_loader:
        out = model(x_val)
        preds = F.log_softmax(out, dim=1).argmax(dim=1)
        total += y_val.size(0)
        correct += (preds == y_val).sum().item()
    
    acc = correct / total

    if epoch % 500 == 0:
        print(f'Epoch: {epoch:3d}. Loss: {loss.item():.4f}. Acc.: {acc:2.2%}')

    if acc > best_acc:
        trials = 0
        best_acc = acc
        torch.save(model.state_dict(), 'best.pth')
        print(f'Epoch {epoch} best model saved with accuracy: {best_acc:2.2%}')
    else:
        trials += 1
        if trials >= patience:
            print(f'Early stopping on epoch {epoch}')
            break

Start model training
Epoch 1 best model saved with accuracy: 67.91%
Epoch 2 best model saved with accuracy: 72.57%
Epoch 3 best model saved with accuracy: 75.00%
Epoch 4 best model saved with accuracy: 76.28%
Epoch 5 best model saved with accuracy: 77.24%
Epoch 6 best model saved with accuracy: 78.08%
Epoch 7 best model saved with accuracy: 78.90%
Epoch 8 best model saved with accuracy: 79.48%
Epoch 9 best model saved with accuracy: 79.76%
Epoch 10 best model saved with accuracy: 79.90%
Epoch 12 best model saved with accuracy: 79.93%
Epoch 13 best model saved with accuracy: 79.94%
Epoch 18 best model saved with accuracy: 80.28%
Epoch 19 best model saved with accuracy: 80.45%
Epoch 20 best model saved with accuracy: 80.84%
Epoch 23 best model saved with accuracy: 80.87%
